# Задание по программированию: 1NN против RandomForest

Курс "Обучение на размеченных данных" https://www.coursera.org/learn/supervised-learning/home/welcome

Автор: Сметанин Александр

Дата: 09.06.2018

## Введение

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Write answers into a file
def write_answer(filename, results):
    with open(filename, "w") as fout:
        fout.write(" ".join([str(res) for res in results]))

In [3]:
ds_digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(ds_digits.data, 
                                                    ds_digits.target, 
                                                    test_size=0.25, 
                                                    random_state=3634, 
                                                    shuffle = True)


Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [4]:
# Функция, вычисляющая евклидово расстояние между точками. Параметры a и b - числовые векторы равной длины
def euc_dist(a, b):
    try: 
        return np.sqrt(sum(map(lambda x,y: (x-y)**2, a, b)))
    except TypeError:
        print 'Euc_dist(): TypeError!'
        return None

In [5]:
# Функция, вычисляющая расстояние от точки point до всех точек массива matrix при помощи фукции fun.
# Размерности point и элементов matrix должны совпадать.
# Результат: список пар (расстояние, класс = target)
def dist_list(point, data, target, fun):
    res = list()
    for dat, tgt in zip(data, target):
        d = fun(point, dat)
        res.append([d, tgt])
    return res

In [6]:
# Функция, присваивающая класс объекту методом 1NN.
# test_point - объект
# train_data, train_target - матрица обучающей выборки, обучающий вектор классов
# fun - функция, вычисляющая расстояние между точками
# Результат: класс объекта
def NN1(test_point, train_data, train_target, fun):
    dists = dist_list(test_point, train_data, train_target, fun)
    dists.sort()
    return dists[0][1]

In [7]:
%%time
# Программа
predict = list()
for obj in X_test:
    predict.append(NN1(obj, X_train, y_train, euc_dist))

Wall time: 37.8 s


Время работы при использовании сортировки списков = 36 сек.

In [8]:
# Вычисляем долю ошибок и записываем ее в файл
fault_rate = 1 - metrics.accuracy_score(y_test, predict)
print fault_rate
# write_answer('Week_5_Task_3_Answer_1.txt', [fault_rate])

0.013333333333333308


In [9]:
# Покажем метрики качества
print 'Accuracy =', metrics.accuracy_score(y_test, predict)
print 'Precision =', metrics.precision_score(y_test, predict, average='macro')
print 'Recall =', metrics.recall_score(y_test, predict, average='macro')
print 'F-metrics =', metrics.f1_score(y_test, predict, average='macro')
print '\nConfusion matrix =\n', metrics.confusion_matrix(y_test, predict)
print '\nClassification report\n', metrics.classification_report(y_test, predict)

Accuracy = 0.9866666666666667
Precision = 0.9858281111755535
Recall = 0.9864868421052633
F-metrics = 0.98599478863042

Confusion matrix =
[[42  0  0  0  0  0  0  0  0  0]
 [ 0 40  0  0  0  0  0  0  0  0]
 [ 0  0 41  0  0  0  0  0  0  0]
 [ 0  0  0 60  0  0  0  0  0  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  0  0  0  0 49  0  0  0  1]
 [ 0  0  0  0  0  0 46  0  0  0]
 [ 0  0  0  0  0  0  0 49  0  0]
 [ 0  1  0  0  0  0  1  0 45  1]
 [ 0  0  0  1  1  0  0  0  0 36]]

Classification report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        42
          1       0.98      1.00      0.99        40
          2       1.00      1.00      1.00        41
          3       0.98      1.00      0.99        60
          4       0.97      1.00      0.99        36
          5       1.00      0.98      0.99        50
          6       0.98      1.00      0.99        46
          7       1.00      1.00      1.00        49
          8       1.00      0.

In [10]:
print metrics.classification_report(y_test, predict)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        42
          1       0.98      1.00      0.99        40
          2       1.00      1.00      1.00        41
          3       0.98      1.00      0.99        60
          4       0.97      1.00      0.99        36
          5       1.00      0.98      0.99        50
          6       0.98      1.00      0.99        46
          7       1.00      1.00      1.00        49
          8       1.00      0.94      0.97        48
          9       0.95      0.95      0.95        38

avg / total       0.99      0.99      0.99       450



## Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [11]:
%%time
RF_classifier = RandomForestClassifier(n_estimators=1000, random_state=3634)
RF_classifier.fit(X_train, y_train)
RF_predict = RF_classifier.predict(X_test)

Wall time: 4.29 s


Время работы классификатора RandomForest = 4.46 сек.

In [12]:
# Вычисляем долю ошибок и записываем ее в файл
fault_rate = 1 - metrics.accuracy_score(y_test, RF_predict)
print fault_rate
# write_answer('Week_5_Task_3_Answer_2.txt', [fault_rate])

0.026666666666666616


In [13]:
# Покажем метрики качества
print 'Accuracy =', metrics.accuracy_score(y_test, RF_predict)
print 'Precision =', metrics.precision_score(y_test, RF_predict, average='macro')
print 'Recall =', metrics.recall_score(y_test, RF_predict, average='macro')
print 'F-metrics =', metrics.f1_score(y_test, RF_predict, average='macro')
print '\nConfusion matrix =\n', metrics.confusion_matrix(y_test, RF_predict)
print '\nClassification report\n', metrics.classification_report(y_test, RF_predict)

Accuracy = 0.9733333333333334
Precision = 0.9729022078745265
Recall = 0.9743059371336485
F-metrics = 0.9734090807790384

Confusion matrix =
[[41  0  0  0  1  0  0  0  0  0]
 [ 0 40  0  0  0  0  0  0  0  0]
 [ 1  0 40  0  0  0  0  0  0  0]
 [ 0  0  0 57  0  1  0  0  1  1]
 [ 0  0  0  0 35  0  0  1  0  0]
 [ 0  0  0  0  0 50  0  0  0  0]
 [ 0  0  0  0  0  1 45  0  0  0]
 [ 0  0  0  0  1  0  0 48  0  0]
 [ 0  1  0  1  0  0  0  0 45  1]
 [ 0  0  0  0  0  0  0  1  0 37]]

Classification report
             precision    recall  f1-score   support

          0       0.98      0.98      0.98        42
          1       0.98      1.00      0.99        40
          2       1.00      0.98      0.99        41
          3       0.98      0.95      0.97        60
          4       0.95      0.97      0.96        36
          5       0.96      1.00      0.98        50
          6       1.00      0.98      0.99        46
          7       0.96      0.98      0.97        49
          8       0.98      

## Выводы

На выбранном наборе данных точность выше у метода 1NN, а скорость работы - у классификатора RandomForest.

Качество обоих методов значительно повышается, если перемешать данные перед разделением на обучающую и тестовую выборки. Точность 1NN - до 0,987, RandomForest - до 0,973.